### Example 4.2: Jack’s Car Rental 

##### Jack manages two locations for a nationwide car rental company. 
- Each day, some number of customers arrive at each location to rent cars. If Jack has a car available, he rents it out and is credited \\$10 by the national company.
- If he is out of cars at that location, then the business is lost. 
- Cars become available for renting the day after they are returned. 
- To help ensure that cars are available where they are needed, Jack can move them between the two locations overnight, at a cost of \\$2 per car moved. 

We assume that the number of cars requested and returned at each
location are Poisson random variables, meaning that the probability that the number is
n is $\frac{\lambda^n}{n!}$ $e^{-\lambda}$, where $\lambda$ is the expected number. 
Suppose $\lambda$ is 3 and 4 for rental requests at the first and second locations and 3 and 2 for returns. 
To simplify the problem slightly, we assume that there can be no more than 20 cars at each location 
(any additional cars are returned to the nationwide company, and thus disappear from the problem) and a
maximum of five cars can be moved from one location to the other in one night. 

We take the discount rate to be $\gamma$ = 0.9 and formulate this as a continuing finite MDP, where
the time steps are days, the state is the number of cars at each location at the end of
the day, and the actions are the net numbers of cars moved between the two locations
overnight.

<img src="4.2.png">
Figure 4.2: The sequence of policies found by policy iteration on Jack’s car rental problem,
and the final state-value function. The first five diagrams show, for each number of cars at
each location at the end of the day, the number of cars to be moved from the first location to
the second (negative numbers indicate transfers from the second location to the first). Each
successive policy is a strict improvement over the previous policy, and the last policy is optimal.

In [46]:
import math

In [47]:
class Agent:
    def backup_action(self, env, n1, n2, a):
        """
        Return backup action
        """
        
        # Action a 
        a = max(-n2, min(a, n1))
        
        a = max(-5, min(5, a))
        
        # Value function expected return
        value = -2 * abs(a)
        
        # Start of the day at location 1
        morning_n1 = n1 - a
        
        # Start of the day at location 2
        morning_n2 = n2 + a
        
        # Loop through states
        for (new_n1, new_n2) in env.S:
                
            # Backup update
            value += env.P1[morning_n1][new_n1] * env.P2[morning_n2][new_n2] * (
                env.R1[morning_n1] + env.R2[morning_n2] + env.gamma*env.V[new_n1][new_n2])
                
        return value

In [48]:
class Environment:
    def __init__(self):
        self.size1 = 21
        self.size2 = 26
        self.V = [[0] * self.size1] * self.size1
        self.lambda_requests1 = 3
        self.lambda_requests2 = 4
        self.lambda_dropoffs1 = 3
        self.lambda_dropoffs2 = 2
        self.policy = [[0] * self.size1] * self.size1
        self.gamma = .9
        self.theta = .000001
        self.epsilon = 0.000000001
        self.P1, self.R1 = self.load_P_and_R(self.lambda_requests1, self.lambda_dropoffs1)
        self.P2, self.R2 = self.load_P_and_R(self.lambda_requests2, self.lambda_dropoffs2)
        self.S = list({(i//self.size1, i%self.size1): i%self.size1 for i in range(self.size1*self.size1)}.keys())
        
    def factorial(self, n):
        """
        Return factorial of n
        """
        
        # Base case
        if n == 0: return 1
        
        # Recursive call
        return n * self.factorial(n-1)
    
    def poisson (self, n, lambdas):
        """
        The probability of n events according to the poisson distribution
        """
        
        return math.exp(-lambdas) * (lambdas**n/self.factorial(n))
    
    def load_P_and_R(self, lambda_requests, lambda_dropoffs):
        """
        Load P(s'|s,a) and R(s,a)
        """
        # Dynamics P
        P = [[0] * self.size1] * self.size2
        
        # Rewards
        R = [0] * self.size2
        
        # Requests and dropoffs per day
        requests = dropoffs = 0
        
        # Request and dropoff probability
        request_prob = drop_prob = self.theta
        
        # Load requests and dropoffs until probability is near certain
        while request_prob >= self.theta:
            
            # Poisson random variable for request
            request_prob = self.poisson(requests, lambda_requests)
            
            # Get transitions for requests
            for n in range(self.size2):
                
                # Load reward for request
                R[n] += 10 * request_prob * min(requests, n)
            
            # Load dropoffs until probability is near certain
            while drop_prob >= self.theta:
                
                # Poisson random variable for dropoff
                drop_prob = self.poisson(dropoffs, lambda_dropoffs)
                    
                # Get transitions for dropoffs
                for n in range(self.size2):

                    # Number of satisfied requests
                    satisfied_requests = min(requests, n)

                    # index Probability of s', r
                    new_n = max(0, min(20, (n + dropoffs) - satisfied_requests))

                    # Load Dynamic function P
                    P[n][new_n] += request_prob * drop_prob
                
                # dropoffs
                dropoffs += 1
    
            # Request
            requests += 1
                    
        return P, R
    
    def pi(self, n1, n2, agent):
        """
        Policy pi(a|s)
        """
        
        # Best value
        best_value = 0
        
        # current values
        this_value = float('-inf')
        
        # Best action in state
        best_action = 0
        
        # Check range of actions
        for a in range(max(-5, -n2), min(5, n1)+1):
            
            # Get the best value and action from state
            while this_value <= best_value + self.epsilon:
                
                # Bellman backup update
                this_value = agent.backup_action(self, n1, n2, a)
            
            # Best action value
            best_value = this_value
            
            # Optimal action
            best_action = a
            
        return best_action

In [49]:
def policy_evaluation(env, agent):
    """
    Return v_pi after policy evaluation
    """
    
    # Delta < theta condition
    delta = env.theta
    
    # Loop until delta < theta
    while delta >= env.theta:
        
        # Delta <-- 0
        delta = 0
        
        # Loop for each s in S
        for (s1, s2) in env.S:
            
            # v <-- V(s)
            v = env.V[s1][s2]
            
            # Bellman update
            env.V[s1][s2] = agent.backup_action(env, s1, s2, env.policy[s1][s2])
            
            # delta <-- max(delta, |v-V(s)|)
            delta = max(delta, abs(v-env.V[s1][s2]))
            #print(f"d:{delta}, v:{v} - V:{env.V[s1][s2]} = {abs(v-env.V[s1][s2])}")
    return env.V

In [50]:
def greedify(env, agent):
    """
    Greedify Policy
    """

    # Policy stable or improved 
    policy_improved = True
    
    # Loop for each s in S
    for (n1, n2) in env.S:
        
        # behavior policy
        b = env.policy[n1][n2]
        
        # Improve target policy pi 
        env.policy[n1][n2] = env.pi(n1, n2, agent)  
        
        # Check if policy stable
        if b != env.policy[n1][n2]: policy_improved = False
            
    # policy is stable
    return env.policy, policy_improved

In [51]:
def policy_iteration(env, agent):
    """
    Improve policy and return v* and pi*
    """
    
    # policy stable 
    policy_stable = False
    
    # Until stable
    while not policy_stable:  
            
        # policy evaluation
        env.V = policy_evaluation(env, agent)
        
        # policy improvement
        env.policy, policy_stable = greedify(env, agent)
    
    # Policy stable return v* and pi*
    return env.V, env.policy

In [52]:
if __name__ == "__main__":
    
    # Agent: Employee
    agent = Agent()
    
    # Jack's Car Rental Environment
    env = Environment()
    
    # Policy Iteration on Car Rental
#     v, pi = policy_iteration(env, agent)
    
#     print(v)
#     print(pi)
    print(policy_evaluation(env, agent))

KeyboardInterrupt: 